# 4100 Assignment 1: A*, Heuristics, and the Fifteen Puzzle

In this assignment, you’ll get some experience abstracting a problem into a search problem, implement the A* search algorithm (still the most common way of doing pathfinding in video games), and experiment with the effects of using different heuristics for the search.

You will only need to submit this completed .ipynb notebook to Canvas, as well as a PDF version in case the .ipynb has a problem (Print->Save to PDF).  Despite the redundancy, please make sure you submit the most recent version of your notebook file.

The goal of a fifteen puzzle is to get all fifteen tiles in order from left to right, top to bottom, like so:

1 2 3 4

5 6 7 8

9 10 11 12

13 14 15 -

The only legal moves are to move a tile adjacent to the blank into the blank space, making the tile’s previous space blank. Thus the maximum number of neighbors is 4, but the number of neighbors could be as small as 2 if the blank is in a corner.

**1) Is a count of number of tiles out of place an admissible heuristic if the blank is not counted as a tile? What if the blank is counted as a tile? In each case, if the heuristic is admissible, explain how you know, and if it is not, give an example that shows the heuristic is inadmissible.**




1. Number of tiles displaced given that blank tile is **not** counted is an ADMISSIBLE heuristic because if we have a displaced tile, it takes >=1 moves to relocate to its right place. Given 2 for example, it would take >=2 moves to get the solution.

2. Number of tiles displaced given that blank tile **is** counted is an INADMISSIBLE heuristic because when the solution is off by only 1 move, i.e. 15 is moved from above example, our heuristic would say 2 (counting the blank) which is NOT OPTIMISTIC, given that it can be solved in 1 move.

Here is some provided code to get you started.  Notice the functions that are available to you.

In [34]:
"""Use A* to solve fifteen puzzle instances.

The "main" of this code is solve_and_print, at the end.  We'll try two different
heuristics, counting tiles out of place and summing Manhattan distance from
the destination over all tiles (the better heuristic)."""

import sys
import copy
import numpy as np
from queue import PriorityQueue

PUZZLE_WIDTH = 4
BLANK = 0  # Integer comparison tends to be faster than string comparison

def read_puzzle_string(puzzle_string):
    """Read a NumberPuzzle from string representation; space-delimited, blank is "-".

    Args:
      puzzle_string (string):  string representation of the puzzle

    Returns:
      A NumberPuzzle
    """
    new_puzzle = NumberPuzzle()
    row = 0
    for line in puzzle_string.splitlines():
        tokens = line.split()
        for i in range(PUZZLE_WIDTH):
            if tokens[i] == '-':
                new_puzzle.tiles[row][i] = BLANK
                new_puzzle.blank_r = row
                new_puzzle.blank_c = i
            else:
                try:
                    new_puzzle.tiles[row][i] = int(tokens[i])
                except ValueError:
                    sys.exit("Found unexpected non-integer for tile value")
        row += 1
    return new_puzzle

class NumberPuzzle(object):
    """ Class containing the state of the puzzle, as well as A* bookkeeping info.

    Attributes:
        tiles (numpy array): 2D array of ints for tiles.
        blank_r (int):  Row of the blank, for easy identification of neighbors
        blank_c (int):  Column of blank, same reason
        parent (NumberPuzzle):  Reference to previous puzzle, for backtracking later
        dist_from_start (int):  Steps taken from start of puzzle to here
        key (int or float):  Key for priority queue to determine which puzzle is next
    """

    def __init__(self):
        """ Just return zeros for everything and fill in the tile array later"""
        self.tiles = np.zeros((PUZZLE_WIDTH, PUZZLE_WIDTH))
        self.blank_r = 0
        self.blank_c = 0
        # This next field is for our convenience when generating a solution
        # -- remember which puzzle was the move before
        self.parent = None
        self.dist_from_start = 0
        self.key = 0

    def __str__(self):
        """This is the Python equivalent of Java's toString()."""
        out = ""
        for i in range(PUZZLE_WIDTH):
            for j in range(PUZZLE_WIDTH):
                if j > 0:
                    out += " "
                if self.tiles[i][j] == BLANK:
                    out += "-"
                else:
                    out += str(int(self.tiles[i][j]))
            out += "\n"
        return out

    def copy(self):
        """Copy the puzzle and update the parent field.
        
        In A* search, we generally want to copy instead of destructively alter,
        since we're not backtracking so much as jumping around the search tree.
        Also, if A and B are numpy arrays, "A = B" only passes a reference to B.
        We'll also use this to tell the child we're its parent."""
        child = NumberPuzzle()
        child.tiles = np.copy(self.tiles)
        child.blank_r = self.blank_r
        child.blank_c = self.blank_c
        child.dist_from_start = self.dist_from_start
        child.parent = self
        return child

    def __eq__(self, other):
        """Governs behavior of ==.
        
        Overrides == for this object so that we can compare by tile arrangement
        instead of reference.  This is going to be pretty common, so we'll skip
        a type check on "other" for a modest speed increase"""
        return np.array_equal(self.tiles, other.tiles)

    def __hash__(self):
        """Generate a code for hash-based data structures.
        
        Hash function necessary for inclusion in a set -- unique "name"
        for this object -- we'll just hash the bytes of the 2D array"""
        return hash(bytes(self.tiles))

    def __lt__(self, obj):
        """Governs behavior of <, and more importantly, the priority queue.
        
        Override less-than so that we can put these in a priority queue
        with no problem.  We don't want to recompute the heuristic here,
        though -- that would be too slow to do it every time we need to
        reorganize the priority queue"""
        return self.key < obj.key

    def total_h(self, better_h):
        """A* cost:  admissible heuristic plus cost-so-far.

        Args:
            better_h (boolean):  True for Manhattan distance, false for counting tiles.
          
        Returns:
            A number representing the heuristic value (int or float)
        """
        return self.dist_from_start + self.heuristic(better_h)

    def move(self, tile_row, tile_column):
        """Move from the row, column coordinates given into the blank.

        Also very common, so we will also skip checks for legality to improve speed.

        Args:
            tile_row (int):  Row of the tile to move.
            tile_column (int):  Column of the tile to move.
        """

        self.tiles[self.blank_r][self.blank_c] = self.tiles[tile_row][tile_column]
        self.tiles[tile_row][tile_column] = BLANK
        self.blank_r = tile_row
        self.blank_c = tile_column
        self.dist_from_start += 1

    def legal_moves(self):
        """Return a list of NumberPuzzle states that could result from one move.

        Return a list of NumberPuzzle states that could result from one move
        on the present board.  Use this to keep the order in which
        moves are evaluated the same as our solution, thus matching the
        HackerRank solution as well.  (Also notice we're still in the
        methods of NumberPuzzle, hence the lack of arguments.)

        Returns:
            List of NumberPuzzles.
        """
        legal = []
        if self.blank_r > 0:
            down_result = self.copy()
            down_result.move(self.blank_r-1, self.blank_c)
            legal.append(down_result)
        if self.blank_c > 0:
            right_result = self.copy()
            right_result.move(self.blank_r, self.blank_c-1)
            legal.append(right_result)
        if self.blank_r < PUZZLE_WIDTH - 1:
            up_result = self.copy()
            up_result.move(self.blank_r+1, self.blank_c)
            legal.append(up_result)
        if self.blank_c < PUZZLE_WIDTH - 1:
            left_result = self.copy()
            left_result.move(self.blank_r, self.blank_c+1)
            legal.append(left_result)
        return legal

    def solve(self, better_h):
        """Return a list of puzzle states from this state to solved.

        Args:
            better_h (boolean):  True if Manhattan heuristic, false if tile counting

        Returns:
            path (list of NumberPuzzle or None) - path from start state to finish state
            explored - total number of nodes pulled from the priority queue
        """
        pq = PriorityQueue()
        pq.put((self.total_h(better_h), self))
        c = set()
        count = 0
        while not pq.empty():
          a_key, a = pq.get()
          count += 1
          if a in c:
            continue
          if a.solved():
            return a.path_to_here(), count
          c.add(a)
          for n in a.legal_moves():
            pq.put((n.total_h(better_h), n))

        return None, 0

    def solved(self):
        """"Return True iff all tiles in order and blank in bottom right."""
        should_be = 1
        for i in range(PUZZLE_WIDTH):
            for j in range(PUZZLE_WIDTH):
                if self.tiles[i][j] != should_be:
                    return False
                should_be = (should_be + 1) % (PUZZLE_WIDTH ** 2)
        return True

    def heuristic(self, better_h):
        """Wrapper for the two heuristic functions.

        Args:
            better_h (boolean):  True if Manhattan heuristic, false if tile counting

        Returns:
            Value of the cost-to-go heuristic (int or float)
        """
        if better_h:
            return self.manhattan_heuristic()
        return self.tile_mismatch_heuristic()

    def tile_mismatch_heuristic(self):
        """Returns count of tiles out of place.
        
        Can't count the blank or it's inadmissible."""
        mismatch_count = 0
        should_be = 1
        for i in range(PUZZLE_WIDTH):
            for j in range(PUZZLE_WIDTH):
                if self.tiles[i][j] != BLANK and self.tiles[i][j] != should_be:
                    mismatch_count += 1
                should_be = (should_be + 1) % (PUZZLE_WIDTH ** 2)
        return mismatch_count

    def manhattan_heuristic(self):
        """Returns total Manhattan (city block) distance from destination over all tiles.

        Again, shouldn't count blank; it gets where it's going for free."""
        total_manhattan = 0
        for i in range(PUZZLE_WIDTH):
            for j in range(PUZZLE_WIDTH):
                val = self.tiles[i][j]
                if val != 0:
                    total_manhattan += abs((val - 1) // PUZZLE_WIDTH - i) + abs((val - 1) % PUZZLE_WIDTH - j)
        return total_manhattan

    def path_to_here(self):
        """Returns list of NumberPuzzles giving the move sequence to get here.
        
        Retraces steps to this node through the parent fields."""
        path = []
        current = self
        while not current is None:
            path.insert(0, current)  # push
            current = current.parent
        return path

def print_steps(path):
    """ Print every puzzle in the path.

    Args:
        path (list of NumberPuzzle): list of puzzle states from start to finish
    """
    if path is None:
        print("No path found")
    else:
        print("{} steps".format(len(path)-1))
        for state in path:
            print(state)


def solve_and_print(puzzle_string : str, better_h : bool) -> None:
  """ "Main" - prints series of moves necessary to solve puzzle.

  Args:
    puzzle_string (string):  The puzzle to solve.
    better_h (boolean):  True if Manhattan distance heuristic, false if tile count
  """
  my_puzzle = read_puzzle_string(puzzle_string)
  solution_steps, explored = my_puzzle.solve(better_h)
  print("{} nodes explored".format(explored))
  print_steps(solution_steps)

**2)  In solve(), implement A***, using a heuristic of “number of tiles in the wrong place” as the optimistic estimate of moves to go. (Treat the blank the way you decided was better in question 1.) You will need to make use of two important data structures:

• The queue of puzzle states to explore should be a PriorityQueue, already imported for you at the top. The __lt__() function for NumberPuzzle objects has already been overridden so that it compares the key field to decide what goes first, but that field is currently never initialized. (You'll want it to store the cost-to-go plus heuristic value.)

• Use a set() to efficiently implement a "closed list" of states that have already been explored. (Do not literally use a list, since scanning a list for an item is not efficient.) Sets are hash tables, and the hashing behavior has already been implemented to work in an acceptable way.

solve() should return a list of NumberPuzzles that show the states from the beginning to the end, as well as an integer count of the number of nodes explored (that is, pulled from the front of the priority queue).  The latter is to help you debug and help us grade, although there is some "wiggle room" for reasonable differences in implementation.

Note that you may be penalized if you unnecessarily change the provided code. In particular, you must generate neighbors using the provided legal_moves() function, so that your output should match our own if the heuristics are implemented correctly.

When you have an implementation, try your solution on the provided zero_moves, one_move, and six_moves puzzles using solve_and_print(), and check that they are solved in the required number of moves.

In [35]:
zero_moves = """1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -"""

one_move = """1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15"""

six_moves = """1 2 3 4
5 10 6 8
- 9 7 12
13 14 11 15"""

sixteen_moves = """10 2 4 8
1 5 3 -
9 7 6 12
13 14 11 15"""

forty_moves = """4 3 - 11
2 1 6 8
13 9 7 15
10 14 12 5"""



In [36]:
solve_and_print(zero_moves, False)

1 nodes explored
0 steps
1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -



In [37]:
solve_and_print(one_move, False)

2 nodes explored
1 steps
1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -



In [38]:
solve_and_print(six_moves, False)

7 nodes explored
6 steps
1 2 3 4
5 10 6 8
- 9 7 12
13 14 11 15

1 2 3 4
5 10 6 8
9 - 7 12
13 14 11 15

1 2 3 4
5 - 6 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 - 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 - 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -



**3) Now time your implementation on sixteen_moves**, using the handy Google Colab syntax demonstrated here.

In [39]:
%time solve_and_print(sixteen_moves, False)

398 nodes explored
16 steps
10 2 4 8
1 5 3 -
9 7 6 12
13 14 11 15

10 2 4 -
1 5 3 8
9 7 6 12
13 14 11 15

10 2 - 4
1 5 3 8
9 7 6 12
13 14 11 15

10 - 2 4
1 5 3 8
9 7 6 12
13 14 11 15

- 10 2 4
1 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
- 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
5 - 3 8
9 7 6 12
13 14 11 15

1 - 2 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 - 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 - 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 6 8
9 7 - 12
13 14 11 15

1 2 3 4
5 10 6 8
9 - 7 12
13 14 11 15

1 2 3 4
5 - 6 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 - 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 - 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -

CPU times: user 62.2 ms, sys: 1.02 ms, total: 63.2 ms
Wall time: 68.6 ms


**4) The Manhattan distance of a tile from its final location is the sum of the difference in rows and the difference in columns. If the blank does not count as a tile, does the sum of Manhattan distances from their final locations over all tiles act as an admissible heuristic? What if the blank does count as a tile? In each case, if the heuristic is admissible, explain how you know, and if it is not, give an example that shows the heuristic is inadmissible.**


1. The heuristic is admissible if the blank tile is not counted, because the tiles move vertically or horizontally, so the minimum number of moves required for the tile to get to the position is its Manhattan distance. It could also be greater, but that makes the heuristic more optimistic all the while being more accurate than displaced tiles.
2. The heuristic is inadmissible if we count the blank tile, because the blank tile "adds up" the Manhattan distance lost by the tile while in motion, or vice versa. As an example, if 13,14,15 were to be shifted to right, The heuristic would output a 6, which is even worse compared to the inadmissible version of the displacement heuristic (4). 

**5)** Now **implement Manhattan distance as a new heuristic** in the same block of code above.  Keep your old heuristic, but have the code use the old heuristic if the better_h argument is False, and use the new heuristic if better_h is True.  When you are done, time the new code.

In [40]:
%time solve_and_print(sixteen_moves, True)

68 nodes explored
16 steps
10 2 4 8
1 5 3 -
9 7 6 12
13 14 11 15

10 2 4 -
1 5 3 8
9 7 6 12
13 14 11 15

10 2 - 4
1 5 3 8
9 7 6 12
13 14 11 15

10 - 2 4
1 5 3 8
9 7 6 12
13 14 11 15

- 10 2 4
1 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
- 5 3 8
9 7 6 12
13 14 11 15

1 10 2 4
5 - 3 8
9 7 6 12
13 14 11 15

1 - 2 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 - 4
5 10 3 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 - 8
9 7 6 12
13 14 11 15

1 2 3 4
5 10 6 8
9 7 - 12
13 14 11 15

1 2 3 4
5 10 6 8
9 - 7 12
13 14 11 15

1 2 3 4
5 - 6 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 - 8
9 10 7 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 - 12
13 14 11 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 - 15

1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 -

CPU times: user 22.5 ms, sys: 2.01 ms, total: 24.5 ms
Wall time: 27.4 ms


**6)** Your code should now also finish within two minutes for forty_moves.  **Run it here** to demonstrate.

In [41]:
%time solve_and_print(forty_moves, True)

402443 nodes explored
40 steps
4 3 - 11
2 1 6 8
13 9 7 15
10 14 12 5

4 - 3 11
2 1 6 8
13 9 7 15
10 14 12 5

- 4 3 11
2 1 6 8
13 9 7 15
10 14 12 5

2 4 3 11
- 1 6 8
13 9 7 15
10 14 12 5

2 4 3 11
1 - 6 8
13 9 7 15
10 14 12 5

2 4 3 11
1 9 6 8
13 - 7 15
10 14 12 5

2 4 3 11
1 9 6 8
13 7 - 15
10 14 12 5

2 4 3 11
1 9 - 8
13 7 6 15
10 14 12 5

2 4 - 11
1 9 3 8
13 7 6 15
10 14 12 5

2 - 4 11
1 9 3 8
13 7 6 15
10 14 12 5

- 2 4 11
1 9 3 8
13 7 6 15
10 14 12 5

1 2 4 11
- 9 3 8
13 7 6 15
10 14 12 5

1 2 4 11
9 - 3 8
13 7 6 15
10 14 12 5

1 2 4 11
9 7 3 8
13 - 6 15
10 14 12 5

1 2 4 11
9 7 3 8
13 6 - 15
10 14 12 5

1 2 4 11
9 7 3 8
13 6 12 15
10 14 - 5

1 2 4 11
9 7 3 8
13 6 12 15
10 14 5 -

1 2 4 11
9 7 3 8
13 6 12 -
10 14 5 15

1 2 4 11
9 7 3 -
13 6 12 8
10 14 5 15

1 2 4 -
9 7 3 11
13 6 12 8
10 14 5 15

1 2 - 4
9 7 3 11
13 6 12 8
10 14 5 15

1 2 3 4
9 7 - 11
13 6 12 8
10 14 5 15

1 2 3 4
9 7 11 -
13 6 12 8
10 14 5 15

1 2 3 4
9 7 11 8
13 6 12 -
10 14 5 15

1 2 3 4
9 7 11 8
13 6 - 12
10 14 

**7) Suppose you decide to try out Euclidean distance, $\sqrt{r^2 + c^2}$ where r and c are the row and column differences, as a heuristic.  It runs faster than tiles displaced, but slower than Manhattan distance.  Why?  (Assume it's not the slowness of square root operations, or anything like that.)**


The Heuristic mentioned above, calculates the physical distance between the current tile location to its actual position. While it's admissible, it can be slower because it is not as accurate as Manhattan distance. This is because the player can move the tiles either horizontally or vertically but not diagonally, but the hueristic actually calculates the diagonal distance. if a tile is 1 up and 1 left of its actual place, the manhattan distance is 2 which is more accurate to reality than 1.414. 

**8) Suppose a bug caused you to calculate the Manhattan distance incorrectly, so that it only returned the number of rows away for each tile, ignoring columns.  Is this heuristic still going to return an optimal solution every time?  Why or why not?**


It will return an optimal solution every time, for it is an admissible heuristic. (because the value always is going to be lesser or equal to the manhattan distance) However, it would be inefficient, and would take a longer time to process as it is not as accurate as the manhattan distance.

**9) In some fifteen-puzzle implementations, you can slide not just one tile, but all tiles to one side of the blank into the blank space.  (For example, if the bottom row were - 13 14 15, one move could cause 13 14 15 -.) Are the tiles displaced and Manhattan distance heuristics admissible in that case?  If so, explain how you know.  If not, provide a heuristic that is admissible and can return at least 4 different values, and explain how you know it is admissible.  (Heuristics should not return values less than 0.  Hint:  Consider simple arithmetic to modify the existing heuristics.)**

The tiles-displaced and the manhattan heuristic are rendered useless by the new implementation, as 3 tiles at maximum can be put into their respective places in 1 move < 3. The best alteration to modify the existing heuristic is to take either the manhattan distance or the tiles displaced and divide it by 3 (and floor it).
(i.e. NumberPuzzle().heuristic(bool) // 3). Given the total 15, the answers would range from 0 to 5

A* is one of the most successful algorithms in the history of AI, a champ at what it does (as long as you can come up with a good heuristic), and is probably the most important AI technology for a modern game AI programmer to understand (because of all the pathfinding that goes on). It's a classic technique for a reason!

**Be sure you've done all other bold text, then "File->Download .ipynb" and upload your file to Canvas, along with a PDF version (File->Print->Destination->Save as PDF).**